### What this script does:

- Remove stopwords: English & French stopwords are loaded from two txt files into two sets. When a comment/post is detected as in French, detectable French stopwords will be removed. Otherwise, English stopwords will be removed accordingly. Note: the stopwords list needs to be updated to suit our purpose, for example, when we need to find out about the attitude/sentiment of comments, we should probably exclude words such as 'couldn't', 'cannot' or 'mustn't' from this list. 


- Remove non ascii characters: after this step, only digits, English & French characters are kept. Turn it off if it's unnecessary or removes too much implicit information such as emoji's.


- Tokenization: with MWETokenizer, multi-word tokens can be added based on our needs, for example, I've added 'climate change', 'canada150', 'justin trudeau' as customized tokens and counted their frequency in later step. 


- Stemming: stem the tokens obtained from last step.


- Output tokens' & stemmers' frequency distribution: output csv files can be found in folder 'word_count', frequency of tokens and stemmers are listed in descending order. The freq_perc column is obtained by dividing the frequency of a word by the total number of comment/posts in each original csv file. Note: Note: there are duplicated tokens in each comment/post, we can choose to count the duplicate or not by commenting/uncommenting a line of code.


In [1]:
import glob
import os
import pandas as pd
from langdetect import detect
import nltk
#nltk.download()   # comment after first download
from nltk.tokenize import sent_tokenize, MWETokenizer, wordpunct_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.stem.snowball import EnglishStemmer
from nltk.probability import FreqDist

## Choose a path

In [2]:
#======= Twitter =======
rootPath='../tw/cleaned data/*.csv'
COLUMN_NAME = 'full_text_cleaned'

#======= Facebook =======
#rootPath='../fb/cleaned data/comments/*.csv'
#COLUMN_NAME = 'comment_message_cleaned'

#======= Instagram =======
#rootPath='../in/cleaned data/posts/*.csv'
#COLUMN_NAME = 'caption'

multiWordsPath = './multiwords.txt'

In [3]:
outputDir = rootPath[:-5] + 'word_count/'
if not os.path.exists(outputDir):
    os.makedirs(outputDir)
filePaths = glob.glob(rootPath)  
display(filePaths)

['../tw/cleaned data/ParksCanada_tweets.csv',
 '../tw/cleaned data/ParcsCanada_tweets.csv']

In [4]:
testfile = '../tw/cleaned data/ParksCanada_tweets.csv'
filename = os.path.basename(testfile)
outputFileName = filename[:-4] + '_word_count.csv'
display(outputFileName)


stopWords_en = set(stopwords.words('english'))
stopWords_fr = set(stopwords.words('french'))

'ParksCanada_tweets_word_count.csv'

In [5]:
def detect_lang(text):
    try:
        lang = detect(text)
    except:
        return 'error'
    return lang


def filter_stop_words(text):   
    stopWords = stopWords_en  
    if detect_lang(text) == 'fr':
        stopWords = stopWords_fr  
    filtered_text = [w for w in wordpunct_tokenize(text) if w.lower() not in stopWords
            and len(w) > 1 and w.isalnum()]   
    return ' '.join(filtered_text)


def load_multi_words(filepath):
    with open(filepath) as file:
        lines = file.readlines()
        words = [word.strip() for word in lines]
        return words


def tokenize_multi_words(topic_list):
    result = []
    print('>>>Adding custermized topic words/tokens to tokenizer...')
    for words in topic_list:
        print(words)
        result.append(words.split('_'))
    return result

multiWords = load_multi_words(multiWordsPath)       # load custermized tokens
tokenizedMultiWords = tokenize_multi_words(multiWords)
tokenizer = MWETokenizer(tokenizedMultiWords)
#tokenizer = MWETokenizer()    # Uncomment this line if no customized multi-word tokens needed

def tokenize_text(text):
    return tokenizer.tokenize(text.split())   # remove .lower()


def stem_text(text):
    if detect_lang(text) == 'fr':
        stemmer = FrenchStemmer(ignore_stopwords=False)
    else:
        stemmer = EnglishStemmer(ignore_stopwords=False) 
    stems = [stemmer.stem(tok) for tok in text]
    return stems


>>>Adding custermized topic words/tokens to tokenizer...
climate_change
canada150
justin_trudeau


In [6]:
df = pd.read_csv(testfile)   # the first unnamed column already exists in csv file 
#df
#df[COLUMN_NAME].tolist()

In [7]:
df['text_filtered'] = df[COLUMN_NAME].astype(str).apply(filter_stop_words)
pd.options.display.max_rows = 999
df[[COLUMN_NAME, 'text_filtered']]
#df['text_filtered'].tolist()

,full_text_cleaned,text_filtered
0,Immerse yourself in a true winter wilderness i...,Immerse true winter wilderness inspirational s...
1,? Book early! Our reservations start soon!,Book early reservations start soon
2,"Put a Canadian twist on your plans, and cue up...",Put Canadian twist plans cue stunning music vi...
3,We know you’ve had some great in our this year...,know great year tell us Share best eerience
4,From starry backcountry nights to cozy rustic ...,starry backcountry nights cozy rustic shelters...
5,"Looking for getaway? Four hours from Toronto, ...",Looking getaway Four hours Toronto unwind amon...
6,"our Family/Group is % off until December , ? D...",Family Group December miss opportunity save ne...
7,was full of and and we want to hear about it! ...,full want hear final want show us best eerience
8,"It’s , which means the year is coming to an en...",means year coming end spend much time great ou...
9,Did bring you new gear? The only thing left is...,bring new gear thing left book reservations st...


## Load multi-word tokens, tokenize comments/posts, and stem tokens

In [8]:
df['text_tokenized'] = df['text_filtered'].apply(tokenize_text)
df['text_stemmed'] = df['text_tokenized'].apply(stem_text)

In [9]:
token_lst = df['text_tokenized'].tolist()
token_lst
token_fdist = FreqDist()
for list_i in token_lst:
    list_i = set(list_i)  # Adding this line would count a word once even if it appears multple times in one comment/post
    for token in list_i:
        token_fdist[token.lower()] += 1
#token_fdist.most_common(30)
#token_fdist['justin_trudeau']       # check the frequency of a token

In [10]:
stemmer_lst = df['text_stemmed'].tolist()
stemmer_lst
stemmer_fdist = FreqDist()
for list_i in stemmer_lst:
    list_i = set(list_i)  # Adding this line would count a word once even if it appears multple times in one comment/post
    for token in list_i:
        stemmer_fdist[token.lower()] += 1
#stemmer_fdist.most_common(30)
 
display(stemmer_fdist['canada150']) # stemmer doesn't change multi-word tokens
display(stemmer_fdist['justin_trudeau'])        

0

0

## Output token/stemmer frequency distribution

In [11]:
token_df = pd.DataFrame(list(token_fdist.items()), columns=['token', 'tok_freq'])
token_df['tok_freq_perc'] = token_df.tok_freq/len(df)
token_df = token_df.sort_values('tok_freq', ascending=False).reset_index(drop=True)

stemmer_df = pd.DataFrame(list(stemmer_fdist.items()), columns=['stemmer', 'stem_freq'])
stemmer_df['stem_freq_perc'] = stemmer_df.stem_freq/len(df)
stemmer_df = stemmer_df.sort_values('stem_freq', ascending=False).reset_index(drop=True)

print('>>> Output word frequency distribution for ' + filename)
output_df = pd.concat([token_df, stemmer_df], axis=1)
output_df.to_csv(outputDir + outputFileName, index=None)   
output_df

>>> Output word frequency distribution for ParksCanada_tweets.csv


,token,tok_freq,tok_freq_perc,stemmer,stem_freq,stem_freq_perc
0,us,76,0.097812,park,88.0,0.113256
1,see,67,0.086229,place,79.0,0.101673
2,week,61,0.078507,us,76.0,0.097812
3,places,59,0.075933,see,68.0,0.087516
4,check,58,0.074646,share,65.0,0.083655
5,share,56,0.072072,week,63.0,0.081081
6,get,55,0.070785,site,61.0,0.078507
7,park,54,0.069498,check,60.0,0.077220
8,national,53,0.068211,get,59.0,0.075933
9,canada,51,0.065637,nation,56.0,0.072072
